### WNS Analytics Wizard 2019

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

#importing data
train=pd.read_csv('C:/Users/deepak/Documents/WNS Analytics Wizard 2019/train.csv')
test=pd.read_csv('C:/Users/deepak/Documents/WNS Analytics Wizard 2019/test.csv')
item_data=pd.read_csv('C:/Users/deepak/Documents/WNS Analytics Wizard 2019/item_data.csv')
view_log=pd.read_csv('C:/Users/deepak/Documents/WNS Analytics Wizard 2019/view_log.csv')

In [2]:
train.shape

(237609, 7)

In [41]:
train.user_id.nunique()

74723

In [53]:
train.is_click.value_counts()

0    226747
1     10862
Name: is_click, dtype: int64

In [3]:
test.shape

(90675, 6)

In [42]:
test.user_id.nunique()

34079

In [2]:
#Appending train and test
train['flag']='train'
test['flag']='test'

data=train.append(test)
data.shape

(328284, 8)

In [13]:
data.head()

,app_code,flag,impression_id,impression_time,is_4G,is_click,os_version,user_id
0,422,train,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,0,0.0,old,87862
1,467,train,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,1,1.0,latest,63410
2,259,train,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,1,0.0,intermediate,71748
3,244,train,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,1,0.0,latest,69209
4,473,train,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,0,0.0,latest,62873


In [37]:
data.user_id.nunique()

89157

In [38]:
data.impression_id.nunique()

328284

In [10]:
# No missing value in the main data
data.isnull().sum()

app_code               0
flag                   0
impression_id          0
impression_time        0
is_4G                  0
is_click           90675
os_version             0
user_id                0
dtype: int64

In [11]:
#Lets do basic profiling
import pandas_profiling as pp
pp.ProfileReport(data)

Number of variables,9
Number of observations,328284
Total Missing (%),3.1%
Total size in memory,22.5 MiB
Average record size in memory,72.0 B
Numeric,4
Categorical,3
Boolean,1
Date,0
Text (Unique),1
Rejected,0


#### Exploring item_data

In [16]:
item_data.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


In [19]:
#Checking for outliers
numeric_var_names=[key for key in dict(item_data.dtypes) if dict(item_data.dtypes)[key] in ['float64', 'int64', 'float32', 'int32']]
print(numeric_var_names) 

num=item_data[numeric_var_names]

def var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  x.std(), x.var(), x.min(), x.dropna().quantile(0.01), x.dropna().quantile(0.05),x.dropna().quantile(0.10),x.dropna().quantile(0.25),x.dropna().quantile(0.50),x.dropna().quantile(0.75), x.dropna().quantile(0.90),x.dropna().quantile(0.95), x.dropna().quantile(0.99),x.max()], 
                  index=['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1' , 'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])

num_summary=num.apply(lambda x: var_summary(x)).T

num_summary=num_summary.drop(['item_id'],axis=0)
num_summary


['item_id', 'item_price', 'category_1', 'category_2', 'category_3', 'product_type']


,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
item_price,132761.0,0.0,1.437383e+09,10826.848916,2944.0,28430.189974,8.082757e+08,5.0,154.6,408.0,633.0,1324.0,2944.0,8544.0,24044.0,45337.0,130154.0,1340800.0
category_1,132761.0,0.0,1.244482e+06,9.373852,9.0,5.193278,2.697014e+01,0.0,0.0,1.0,1.0,6.0,9.0,13.0,17.0,17.0,17.0,17.0
category_2,132761.0,0.0,5.194964e+06,39.130196,40.0,21.443609,4.598284e+02,0.0,0.0,3.0,7.0,24.0,40.0,57.0,67.0,74.0,78.0,79.0
category_3,132761.0,0.0,2.244470e+07,169.060929,159.0,95.310870,9.084162e+03,0.0,10.0,20.0,36.0,85.0,159.0,255.0,300.0,315.0,336.0,341.0
product_type,132761.0,0.0,7.040400e+08,5303.063746,5225.0,2961.645034,8.771341e+06,0.0,140.0,668.0,1293.0,2874.0,5225.0,7844.0,9451.0,10019.0,10420.0,10584.0


In [3]:
#Checking the skewness as well
print(item_data.skew())

#Item_price is badly affected with ouliers (Capping the item_price at 99 percentile and creating flag for items above it.)
item_data['Expensive_item']=np.where(item_data['item_price']>130154.0, 1,0)
item_data['item_price']= item_data['item_price'].clip_upper(item_data['item_price'].quantile(0.99)) 
item_data['item_price']= item_data['item_price'].clip_lower(item_data['item_price'].quantile(0.01))

print('Skewness of item_price after capping outlier :',item_data.item_price.skew())

item_data.head()

item_id         0.000011
item_price      9.786528
category_1     -0.241683
category_2     -0.160575
category_3      0.056008
product_type    0.041453
dtype: float64
Skewness of item_price after capping outlier : 3.9864356088163215


,item_id,item_price,category_1,category_2,category_3,product_type,Expensive_item
0,26880,4602.0,11,35,20,3040,0
1,54939,3513.0,12,57,85,6822,0
2,40383,825.0,17,8,279,1619,0
3,8777,2355.0,13,58,189,5264,0
4,113705,1267.0,17,39,151,10239,0


In [4]:
#Dropping item_price and keeping log_transform variable for the same
to_log = ['item_price']
df_log = item_data[to_log].applymap(lambda x: np.log(x+1))
# rename columns
df_log.columns = 'log_' + df_log.columns

#Joining with the item dataset
item_data=pd.concat([item_data, df_log], axis = 1)
item_data.drop(['item_price'],axis=1,inplace=True)

#### Exploring view_log

In [5]:
view_log.shape

(3118622, 5)

In [48]:
view_log.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


In [50]:
view_log.isnull().sum()

server_time    0
device_type    0
session_id     0
user_id        0
item_id        0
dtype: int64

In [51]:
#Heavily skewed by android
view_log.device_type.value_counts()

android    3117737
iphone         861
web             24
Name: device_type, dtype: int64

In [5]:
# Merging item_data with it
view_log=pd.merge(view_log, item_data, on='item_id', how='left')

In [6]:
from datetime import datetime

year = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year
month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month
day = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).day
hour = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).hour
minute = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).minute
second = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).second

view_log['year']=view_log['server_time'].map(year)
view_log['month']=view_log['server_time'].map(month)
view_log['day']=view_log['server_time'].map(day)
view_log['hour']=view_log['server_time'].map(hour)
view_log['minute']=view_log['server_time'].map(minute)
view_log['second']=view_log['server_time'].map(second)

In [7]:
view_log['session_id_str']=view_log.session_id.astype(str)
view_log['product_type_str']=view_log.product_type.astype(str)

In [8]:
view_log.drop(['server_time','year','second','session_id','product_type'],axis=1,inplace=True)

In [9]:
view_log.head()

,device_type,user_id,item_id,category_1,category_2,category_3,Expensive_item,log_item_price,month,day,hour,minute,session_id_str,product_type_str
0,android,4557,32970,16.0,56.0,253.0,0.0,10.909363,10,15,8,58,112333,3184.0
1,android,74788,7640,7.0,71.0,228.0,0.0,7.227662,10,15,8,58,503590,545.0
2,android,23628,128855,4.0,38.0,62.0,0.0,8.421783,10,15,8,58,573960,5609.0
3,android,2430,12774,17.0,39.0,252.0,0.0,6.807935,10,15,8,58,121691,2740.0
4,android,19227,28296,12.0,57.0,54.0,0.0,7.742836,10,15,8,58,218564,7422.0


In [10]:
new_count_data=view_log[['user_id','item_id','Expensive_item','month','day','hour',
                         'minute','session_id_str','product_type_str','category_1','category_2','category_3']]
new_count = new_count_data.groupby(['user_id'], as_index = False).agg(['nunique','count']).reset_index()
# List of column names
columns = ['user_id']
# Iterate through the variables names
for var in new_count.columns.levels[0]:
    # Skip the id name
    if var != 'user_id':
        
        # Iterate through the stat names
        for stat in new_count.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('new_count_%s_%s' % (var, stat))
new_count.columns = columns
new_count=new_count.fillna(new_count.median())


new_sum_data=view_log[['user_id','log_item_price']]
new_sum = new_sum_data.groupby(['user_id'], as_index = False).agg(['sum']).reset_index()
# List of column names
columns = ['user_id']
# Iterate through the variables names
for var in new_sum.columns.levels[0]:
    # Skip the id name
    if var != 'user_id':
        
        # Iterate through the stat names
        for stat in new_sum.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('new_sum_%s_%s' % (var, stat))
new_sum.columns = columns
new_sum=new_sum.fillna(new_sum.median())


data = data.merge(new_count, on = 'user_id', how = 'left')
data = data.merge(new_sum, on = 'user_id', how = 'left')

data.shape

(328284, 31)

In [11]:
data['year']=data['impression_time'].map(year)
data['month']=data['impression_time'].map(month)
data['day']=data['impression_time'].map(day)
data['hour']=data['impression_time'].map(hour)
data['minute']=data['impression_time'].map(minute)
data['second']=data['impression_time'].map(second)

#Creating dummy variables
data=pd.get_dummies(data,columns=['os_version'],drop_first=True)

In [12]:
data.drop(['year','second','impression_time'],axis=1,inplace=True)

In [13]:
add_data=data[['impression_id','app_code','user_id']]
add_data_count = add_data.groupby(['impression_id'], as_index = False).agg(['count']).reset_index()
# List of column names
columns = ['impression_id']
# Iterate through the variables names
for var in add_data_count.columns.levels[0]:
    # Skip the id name
    if var != 'impression_id':
        
        # Iterate through the stat names
        for stat in add_data_count.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('agg_%s_%s' % (var, stat))
add_data_count.columns = columns
add_data_count=add_data_count.fillna(add_data_count.median())

data = data.merge(add_data_count, on = 'impression_id', how = 'left')
data.shape

(328284, 37)

In [14]:
data.head()

,app_code,flag,impression_id,is_4G,is_click,user_id,new_count_item_id_nunique,new_count_item_id_count,new_count_Expensive_item_nunique,new_count_Expensive_item_count,...,new_count_category_3_count,new_sum_log_item_price_sum,month,day,hour,minute,os_version_latest,os_version_old,agg_app_code_count,agg_user_id_count
0,422,train,c4ca4238a0b923820dcc509a6f75849b,0,0.0,87862,1,1,1,1,...,1,7.762596,11,15,0,0,0,1,1,1
1,467,train,45c48cce2e2d7fbdea1afc51c7c6ad26,1,1.0,63410,7,12,1,12,...,12,98.390874,11,15,0,1,1,0,1,1
2,259,train,70efdf2ec9b086079795c442636b55fb,1,0.0,71748,2,2,1,2,...,2,14.588923,11,15,0,2,0,0,1,1
3,244,train,8e296a067a37563370ded05f5a3bf3ec,1,0.0,69209,12,18,1,18,...,18,143.493966,11,15,0,2,1,0,1,1
4,473,train,182be0c5cdcd5072bb1864cdee4d3d6e,0,0.0,62873,34,46,1,46,...,46,372.369547,11,15,0,2,1,0,1,1


In [15]:
data=pd.get_dummies(data,columns=['app_code'],drop_first=True)

In [16]:
data.shape

(328284, 557)

### Additional Dummies from all datasets 

In [17]:
new_dummy_data=view_log[['user_id','month','hour','minute','day']]
new_dummy = new_dummy_data.groupby(['user_id'], as_index = False).agg(['min','max']).reset_index()
# List of column names
columns = ['user_id']
# Iterate through the variables names
for var in new_dummy.columns.levels[0]:
    # Skip the id name
    if var != 'user_id':
        
        # Iterate through the stat names
        for stat in new_dummy.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('dummy_%s_%s' % (var, stat))
new_dummy.columns = columns
new_dummy=new_dummy.fillna(new_dummy.median())


In [18]:
new_dummy.shape

(89157, 9)

In [19]:
new_dummy.head()

,user_id,dummy_month_min,dummy_month_max,dummy_hour_min,dummy_hour_max,dummy_minute_min,dummy_minute_max,dummy_day_min,dummy_day_max
0,0,10,12,6,20,4,54,4,26
1,1,11,12,1,9,6,37,4,9
2,2,10,12,7,23,0,59,1,31
3,3,10,10,15,16,1,52,19,19
4,4,12,12,23,23,19,20,7,7


In [20]:
data = data.merge(new_dummy, on = 'user_id', how = 'left')

In [21]:
data.drop(['user_id'],axis=1,inplace=True)

In [22]:
data['diff_month_max']=data['month']-data['dummy_month_max']
data['diff_month_min']=data['month']-data['dummy_month_min']

data['diff_hour_max']=data['hour']-data['dummy_hour_max']
data['diff_hour_min']=data['hour']-data['dummy_hour_min']

data['diff_minute_max']=data['minute']-data['dummy_minute_max']
data['diff_minute_min']=data['minute']-data['dummy_minute_min']

data['diff_day_max']=data['day']-data['dummy_day_max']
data['diff_day_min']=data['day']-data['dummy_day_min']

In [24]:
data['month_max_ratio']=data['month']/data['dummy_month_max']
data['month_min_ratio']=data['month']/data['dummy_month_min']

data['month_hour_ratio']=data['month']/data['dummy_hour_max']
data['month_hour_ratio']=data['month']/data['dummy_hour_min']

data['month_minute_ratio']=data['month']/data['dummy_minute_max']
data['month_minute_ratio']=data['month']/data['dummy_minute_min']

data['month_day_ratio']=data['month']/data['dummy_day_max']
data['month_day_ratio']=data['month']/data['dummy_day_min']


In [25]:
data=data.replace([np.inf, -np.inf], 0)

In [26]:
data.head()

,flag,impression_id,is_4G,is_click,new_count_item_id_nunique,new_count_item_id_count,new_count_Expensive_item_nunique,new_count_Expensive_item_count,new_count_month_nunique,new_count_month_count,...,diff_hour_min,diff_minute_max,diff_minute_min,diff_day_max,diff_day_min,month_max_ratio,month_min_ratio,month_hour_ratio,month_minute_ratio,month_day_ratio
0,train,c4ca4238a0b923820dcc509a6f75849b,0,0.0,1,1,1,1,1,1,...,-18,-11,-11,-4,-4,1.100000,1.1,0.611111,1.0,0.578947
1,train,45c48cce2e2d7fbdea1afc51c7c6ad26,1,1.0,7,12,1,12,3,12,...,-5,-58,1,-10,5,0.916667,1.1,2.200000,0.0,1.100000
2,train,70efdf2ec9b086079795c442636b55fb,1,0.0,2,2,1,2,2,2,...,-12,-57,2,-8,-8,1.000000,1.1,0.916667,0.0,0.478261
3,train,8e296a067a37563370ded05f5a3bf3ec,1,0.0,12,18,1,18,1,18,...,-2,-57,1,-8,4,1.000000,1.0,5.500000,11.0,1.000000
4,train,182be0c5cdcd5072bb1864cdee4d3d6e,0,0.0,34,46,1,46,3,46,...,0,-54,1,-11,10,0.916667,1.1,0.000000,11.0,2.200000


In [27]:
#Separating train and test
train_1=data[data.flag=='train']
test_1=data[data.flag=='test']

train_x=train_1.drop(['flag','impression_id','is_click'],axis=1)
train_y=train_1[['is_click']]

test_x=test_1.drop(['flag','impression_id','is_click'],axis=1)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(237609, 574)
(237609, 1)
(90675, 574)


In [28]:
# Threshold for removing correlated variables
threshold = 0.9
# Absolute value correlation matrix
corr_matrix = train_1.corr().abs()
corr_matrix.head()

,is_4G,is_click,new_count_item_id_nunique,new_count_item_id_count,new_count_Expensive_item_nunique,new_count_Expensive_item_count,new_count_month_nunique,new_count_month_count,new_count_day_nunique,new_count_day_count,...,diff_hour_min,diff_minute_max,diff_minute_min,diff_day_max,diff_day_min,month_max_ratio,month_min_ratio,month_hour_ratio,month_minute_ratio,month_day_ratio
is_4G,1.000000,0.003464,0.070463,0.074474,0.039038,0.074480,0.083468,0.074474,0.085685,0.074474,...,0.008432,0.031006,0.022958,0.055555,0.012413,0.026374,0.067330,0.017516,0.001871,0.067532
is_click,0.003464,1.000000,0.046157,0.046033,0.019149,0.046029,0.033680,0.046033,0.050350,0.046033,...,0.037152,0.018005,0.024423,0.022185,0.018655,0.033165,0.006213,0.000923,0.001446,0.041521
new_count_item_id_nunique,0.070463,0.046157,1.000000,0.952534,0.445752,0.952511,0.446175,0.952534,0.779648,0.952534,...,0.252484,0.189275,0.182613,0.199854,0.206876,0.244502,0.251238,0.051035,0.116701,0.455245
new_count_item_id_count,0.074474,0.046033,0.952534,1.000000,0.417364,0.999991,0.407652,1.000000,0.756012,1.000000,...,0.238257,0.173306,0.169075,0.184639,0.190606,0.223667,0.229111,0.057852,0.126461,0.432684
new_count_Expensive_item_nunique,0.039038,0.019149,0.445752,0.417364,1.000000,0.417319,0.280069,0.417364,0.444416,0.417364,...,0.155344,0.119872,0.119293,0.130998,0.134991,0.161299,0.148779,0.020483,0.028307,0.267117


In [29]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

,is_4G,is_click,new_count_item_id_nunique,new_count_item_id_count,new_count_Expensive_item_nunique,new_count_Expensive_item_count,new_count_month_nunique,new_count_month_count,new_count_day_nunique,new_count_day_count,...,diff_hour_min,diff_minute_max,diff_minute_min,diff_day_max,diff_day_min,month_max_ratio,month_min_ratio,month_hour_ratio,month_minute_ratio,month_day_ratio
is_4G,NaN,0.003464,0.070463,0.074474,0.039038,0.074480,0.083468,0.074474,0.085685,0.074474,...,0.008432,0.031006,0.022958,0.055555,0.012413,0.026374,0.067330,0.017516,0.001871,0.067532
is_click,NaN,NaN,0.046157,0.046033,0.019149,0.046029,0.033680,0.046033,0.050350,0.046033,...,0.037152,0.018005,0.024423,0.022185,0.018655,0.033165,0.006213,0.000923,0.001446,0.041521
new_count_item_id_nunique,NaN,NaN,NaN,0.952534,0.445752,0.952511,0.446175,0.952534,0.779648,0.952534,...,0.252484,0.189275,0.182613,0.199854,0.206876,0.244502,0.251238,0.051035,0.116701,0.455245
new_count_item_id_count,NaN,NaN,NaN,NaN,0.417364,0.999991,0.407652,1.000000,0.756012,1.000000,...,0.238257,0.173306,0.169075,0.184639,0.190606,0.223667,0.229111,0.057852,0.126461,0.432684
new_count_Expensive_item_nunique,NaN,NaN,NaN,NaN,NaN,0.417319,0.280069,0.417364,0.444416,0.417364,...,0.155344,0.119872,0.119293,0.130998,0.134991,0.161299,0.148779,0.020483,0.028307,0.267117


In [30]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))

There are 22 columns to remove.


In [31]:
train_x = train_x.drop(columns = to_drop)
test_x = test_x.drop(columns = to_drop)

print('Training shape: ', train_x.shape)
print('Testing shape: ', test_x.shape)

Training shape:  (237609, 552)
Testing shape:  (90675, 552)


In [32]:
#Feature importance
from sklearn.ensemble import RandomForestClassifier

In [33]:
feature_importances = np.zeros(train_x.shape[1])

feat_labels=data.columns

clf = RandomForestClassifier(n_jobs=-1)

# Fit the model seven times to avoid overfitting
for i in range(7):
    
    # Train the classifier
    clf.fit(train_x, train_y)

    # Record the feature importances
    feature_importances += clf.feature_importances_

In [34]:
feature_importances = feature_importances / 7
feature_importances = pd.DataFrame({'feature': list(train_x.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)

In [35]:
feature_importances

,feature,importance
546,diff_minute_min,0.075282
545,diff_minute_max,0.075162
7,minute,0.073283
547,diff_day_max,0.055553
548,diff_day_min,0.054052
1,new_count_item_id_nunique,0.049775
544,diff_hour_min,0.049136
543,diff_hour_max,0.048151
6,hour,0.045444
4,new_count_day_nunique,0.041405


In [36]:
feature_importances.to_csv('C:/Users/deepak/Documents/WNS Analytics Wizard 2019/rf_imp.csv')

In [37]:
zero_features = list(feature_importances[feature_importances['importance'] <=0.000157963]['feature'])

train_x = train_x.drop(columns = zero_features)
test_x = test_x.drop(columns = zero_features)

print('Training shape: ', train_x.shape)
print('Testing shape: ', test_x.shape)

Training shape:  (237609, 148)
Testing shape:  (90675, 148)


In [38]:
from sklearn.preprocessing import StandardScaler
Scaler=StandardScaler()
train_x1 = Scaler.fit_transform(train_x)
test_x1 = Scaler.transform(test_x)

print(train_x1.shape)
print(test_x1.shape)

(237609, 148)
(90675, 148)


### Model building

In [40]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

### Logistic Regression

In [54]:
from sklearn.linear_model import LogisticRegression
gamma_range = [10 ** x for x in range(-5, 1)]
weight=[None,'balanced']
penalty=['l1','l2']

param_distribution = dict(C=gamma_range,class_weight=weight,penalty=penalty)
print(param_distribution)

# instantiate and fit the grid
grid = RandomizedSearchCV(LogisticRegression(), param_distribution, scoring='roc_auc', cv=5,return_train_score=False,n_jobs=-1)

{'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1], 'class_weight': [None, 'balanced'], 'penalty': ['l1', 'l2']}


In [ ]:
grid.fit(train_x1, train_y)

# examine the best model
print(grid.best_score_)
print(grid.best_params_)

In [58]:
lr_optimal=LogisticRegression(penalty='l2',class_weight='balanced',C=0.001)

# fitting the model
lr_optimal.fit(train_x1, train_y)

# predict the response
test_y_lr = lr_optimal.predict(test_x1)

#predict prob
train_x1_lr_prob = lr_optimal.predict_proba(train_x1)
test_x1_lr_prob = lr_optimal.predict_proba(test_x1)

In [89]:
submission_new = pd.DataFrame({'impression_id': test['impression_id'],
    'is_click': test_y_lr })

In [90]:
submission_new.to_csv("C:/Users/deepak/Documents/WNS Analytics Wizard 2019/lr.csv", index=False)

### Random Forest

In [41]:
from sklearn.ensemble import RandomForestClassifier
n_estimators=list(range(100,1100,100))
max_depth=list(range(1,51,1))
min_samples_split=list(range(2,51,1))
min_samples_leaf=list(range(1,51,1))
weight=[None,'balanced']

param_distributions = dict(n_estimators=n_estimators,max_depth=max_depth,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,class_weight=weight)
print(param_distributions)

# instantiate and fit the grid
grid = RandomizedSearchCV(RandomForestClassifier(), param_distributions, cv=5, scoring='roc_auc', n_iter=50,return_train_score=False,n_jobs=-1)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], 'class_weight': [None, 'balanced']}


In [42]:
grid.fit(train_x1, train_y)

# examine the best model
print(grid.best_score_)
print(grid.best_params_)

0.7744031039720327
{'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_depth': 33, 'class_weight': None}


In [43]:
rf_optimal=RandomForestClassifier(n_estimators=1000,min_samples_split=10,min_samples_leaf=5,
                                  max_depth=33,class_weight=None)

# fitting the model
rf_optimal.fit(train_x1, train_y)

# predict the response
test_y_rf = rf_optimal.predict(test_x1)

#predict prob
train_x1_rf_prob = rf_optimal.predict_proba(train_x1)
test_x1_rf_prob = rf_optimal.predict_proba(test_x1)

In [45]:
submission_new = pd.DataFrame({'impression_id': test['impression_id'],
    'is_click': test_y_rf })

In [46]:
submission_new.to_csv("C:/Users/deepak/Documents/WNS Analytics Wizard 2019/rf.csv", index=False)

### XGBOOST

In [64]:
#Separating train and test
train_1=data[data.flag=='train']
test_1=data[data.flag=='test']

train_x=train_1.drop(['flag','impression_id','is_click'],axis=1)
train_y=train_1[['is_click']]

test_x=test_1.drop(['flag','impression_id','is_click'],axis=1)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(237609, 574)
(237609, 1)
(90675, 574)


In [66]:
# Threshold for removing correlated variables
threshold = 0.9
# Absolute value correlation matrix
corr_matrix = train_1.corr().abs()
corr_matrix.head()

,is_4G,is_click,new_count_item_id_nunique,new_count_item_id_count,new_count_Expensive_item_nunique,new_count_Expensive_item_count,new_count_month_nunique,new_count_month_count,new_count_day_nunique,new_count_day_count,...,diff_hour_min,diff_minute_max,diff_minute_min,diff_day_max,diff_day_min,month_max_ratio,month_min_ratio,month_hour_ratio,month_minute_ratio,month_day_ratio
is_4G,1.000000,0.003464,0.070463,0.074474,0.039038,0.074480,0.083468,0.074474,0.085685,0.074474,...,0.008432,0.031006,0.022958,0.055555,0.012413,0.026374,0.067330,0.017516,0.001871,0.067532
is_click,0.003464,1.000000,0.046157,0.046033,0.019149,0.046029,0.033680,0.046033,0.050350,0.046033,...,0.037152,0.018005,0.024423,0.022185,0.018655,0.033165,0.006213,0.000923,0.001446,0.041521
new_count_item_id_nunique,0.070463,0.046157,1.000000,0.952534,0.445752,0.952511,0.446175,0.952534,0.779648,0.952534,...,0.252484,0.189275,0.182613,0.199854,0.206876,0.244502,0.251238,0.051035,0.116701,0.455245
new_count_item_id_count,0.074474,0.046033,0.952534,1.000000,0.417364,0.999991,0.407652,1.000000,0.756012,1.000000,...,0.238257,0.173306,0.169075,0.184639,0.190606,0.223667,0.229111,0.057852,0.126461,0.432684
new_count_Expensive_item_nunique,0.039038,0.019149,0.445752,0.417364,1.000000,0.417319,0.280069,0.417364,0.444416,0.417364,...,0.155344,0.119872,0.119293,0.130998,0.134991,0.161299,0.148779,0.020483,0.028307,0.267117


In [67]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

,is_4G,is_click,new_count_item_id_nunique,new_count_item_id_count,new_count_Expensive_item_nunique,new_count_Expensive_item_count,new_count_month_nunique,new_count_month_count,new_count_day_nunique,new_count_day_count,...,diff_hour_min,diff_minute_max,diff_minute_min,diff_day_max,diff_day_min,month_max_ratio,month_min_ratio,month_hour_ratio,month_minute_ratio,month_day_ratio
is_4G,NaN,0.003464,0.070463,0.074474,0.039038,0.074480,0.083468,0.074474,0.085685,0.074474,...,0.008432,0.031006,0.022958,0.055555,0.012413,0.026374,0.067330,0.017516,0.001871,0.067532
is_click,NaN,NaN,0.046157,0.046033,0.019149,0.046029,0.033680,0.046033,0.050350,0.046033,...,0.037152,0.018005,0.024423,0.022185,0.018655,0.033165,0.006213,0.000923,0.001446,0.041521
new_count_item_id_nunique,NaN,NaN,NaN,0.952534,0.445752,0.952511,0.446175,0.952534,0.779648,0.952534,...,0.252484,0.189275,0.182613,0.199854,0.206876,0.244502,0.251238,0.051035,0.116701,0.455245
new_count_item_id_count,NaN,NaN,NaN,NaN,0.417364,0.999991,0.407652,1.000000,0.756012,1.000000,...,0.238257,0.173306,0.169075,0.184639,0.190606,0.223667,0.229111,0.057852,0.126461,0.432684
new_count_Expensive_item_nunique,NaN,NaN,NaN,NaN,NaN,0.417319,0.280069,0.417364,0.444416,0.417364,...,0.155344,0.119872,0.119293,0.130998,0.134991,0.161299,0.148779,0.020483,0.028307,0.267117


In [68]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))

There are 22 columns to remove.


In [69]:
train_x = train_x.drop(columns = to_drop)
test_x = test_x.drop(columns = to_drop)

print('Training shape: ', train_x.shape)
print('Testing shape: ', test_x.shape)

Training shape:  (237609, 552)
Testing shape:  (90675, 552)


In [70]:
from xgboost import XGBClassifier

In [75]:
feature_importances = np.zeros(train_x.shape[1])

feat_labels=data.columns

clf = XGBClassifier(n_jobs=-1)

# Fit the model three times to avoid overfitting
for i in range(7):
    
    # Train the classifier
    clf.fit(train_x, train_y)

    # Record the feature importances
    feature_importances += clf.feature_importances_

In [76]:
feature_importances = feature_importances / 7
feature_importances = pd.DataFrame({'feature': list(train_x.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)

In [78]:
feature_importances

,feature,importance
218,app_code_207,0.090079
396,app_code_386,0.084624
543,diff_hour_max,0.057572
541,diff_month_max,0.051002
544,diff_hour_min,0.047719
547,diff_day_max,0.047243
55,app_code_44,0.046765
548,diff_day_min,0.043194
545,diff_minute_max,0.040485
546,diff_minute_min,0.030516


In [79]:
feature_importances.to_csv('C:/Users/deepak/Documents/WNS Analytics Wizard 2019/xgb_imp.csv')

In [80]:
zero_features = list(feature_importances[feature_importances['importance'] <0.001639053]['feature'])

train_x = train_x.drop(columns = zero_features)
test_x = test_x.drop(columns = zero_features)

print('Training shape: ', train_x.shape)
print('Testing shape: ', test_x.shape)

Training shape:  (237609, 49)
Testing shape:  (90675, 49)


In [81]:
from sklearn.preprocessing import StandardScaler
Scaler=StandardScaler()
train_x1 = Scaler.fit_transform(train_x)
test_x1 = Scaler.transform(test_x)

print(train_x1.shape)
print(test_x1.shape)

(237609, 49)
(90675, 49)


In [82]:
n_estimators=list(range(100,1100,100))
max_depth=list(range(3,30,1))
learning_rate=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
gamma=[0.05,0.1,0.2,0.3,0.4,0.5]
min_child_weight =list(range(1,30,1))
subsample=[0.5,0.6,0.7,0.8,0.9,1.0]
colsample_bytree=[0.5,0.6,0.7,0.8,0.9,1.0]
scale_pos_weight=list(range(1,5,1))

param_distributions  = dict(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate,gamma=gamma,
                           min_child_weight=min_child_weight,subsample=subsample,colsample_bytree=colsample_bytree,
                           scale_pos_weight=scale_pos_weight)
print(param_distributions)

# instantiate and fit the grid
grid = RandomizedSearchCV(XGBClassifier(), param_distributions, cv=5, scoring='roc_auc', return_train_score=False,n_jobs=-1)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'gamma': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5], 'min_child_weight': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'scale_pos_weight': [1, 2, 3, 4]}


In [83]:
grid.fit(train_x1, train_y)

# examine the best model
print(grid.best_score_)
print(grid.best_params_)

0.7414374651777612
{'subsample': 0.6, 'scale_pos_weight': 4, 'n_estimators': 300, 'min_child_weight': 26, 'max_depth': 3, 'learning_rate': 0.8, 'gamma': 0.4, 'colsample_bytree': 0.8}


In [86]:
xgb = XGBClassifier(silent=False, n_jobs=-1, random_state=15, n_estimators=300,max_depth=3,learning_rate=0.8,gamma=0.4,
                            scale_pos_weight=4,subsample=0.6,min_child_weight=26,colsample_bytree=0.8)

# fitting the model
xgb.fit(train_x1, train_y)

# predict the response
test_y_xgb = xgb.predict(test_x1)

#predict prob
train_x1_xgb_prob = xgb.predict_proba(train_x1)
test_x1_xgb_prob = xgb.predict_proba(test_x1)

In [87]:
submission_xgb = pd.DataFrame({'impression_id': test['impression_id'],
    'is_click': test_y_xgb })

In [88]:
submission_xgb.to_csv("C:/Users/deepak/Documents/WNS Analytics Wizard 2019/xgb.csv", index=False)